# import some libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer as vec
from sklearn.svm import LinearSVC
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import cross_val_score
import re
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
wnl = WordNetLemmatizer()
 
print("rocks :", wnl.lemmatize("rocks", pos = 'n'))
print("corpora :", wnl.lemmatize("corpora", pos = 'n'))


rocks : rock
corpora : corpus


# Take datasets from CSV files

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# define function for reprocessing

In [3]:
def extract_alphabetic(text):
    # Use regular expression to remove non-alphabetic characters
    return re.sub(r'[^a-zA-Z\s]', '', text)
def wordnet_lemmatizer(text):
    words = text.split()
    output = [wnl.lemmatize(element) for element in words]
    return ' '.join(output)

In [4]:
df_train['processed text'] = df_train['reviews_content'].apply(extract_alphabetic)
df_train['processed text'] = df_train['processed text'].apply(wordnet_lemmatizer)

df_test['processed text'] = df_test['reviews_content'].apply(extract_alphabetic)
df_test['processed text'] = df_test['processed text'].apply(wordnet_lemmatizer)

# Perform TFIDF

In [6]:
vect = vec(ngram_range = (1,3),min_df = 8,max_df = 1000, norm= 'l2')
vect.fit(df_train['processed text'])
X_train = vect.transform(df_train['processed text'])
y_train = df_train['category']
X_test = vect.transform(df_test['processed text'])

# fine-tuning regularization constant C

In [31]:
for i in np.arange(0.1,2,0.1):
    model = LinearSVC(tol = 0.001, C = i,dual ='auto')
    result = cross_val_score(model, X_train,y_train,cv = 15)
    print(f'C = {i:.2f} avg - {np.mean(result):.2f} median - {np.median(result):.2f}')

C = 0.10 avg - 0.84 median - 0.84
C = 0.20 avg - 0.85 median - 0.86
C = 0.30 avg - 0.86 median - 0.85
C = 0.40 avg - 0.86 median - 0.85
C = 0.50 avg - 0.86 median - 0.86
C = 0.60 avg - 0.86 median - 0.86
C = 0.70 avg - 0.86 median - 0.86
C = 0.80 avg - 0.86 median - 0.86
C = 0.90 avg - 0.87 median - 0.86
C = 1.00 avg - 0.87 median - 0.86
C = 1.10 avg - 0.87 median - 0.86
C = 1.20 avg - 0.87 median - 0.86
C = 1.30 avg - 0.87 median - 0.86
C = 1.40 avg - 0.87 median - 0.86
C = 1.50 avg - 0.87 median - 0.86
C = 1.60 avg - 0.87 median - 0.86
C = 1.70 avg - 0.87 median - 0.86
C = 1.80 avg - 0.87 median - 0.86
C = 1.90 avg - 0.87 median - 0.86


now we choose $C = 0.9$

# Train the model

In [8]:
model = LinearSVC(tol = 0.001, C = 0.9, dual = 'auto')
model.fit(X_train,y_train)
y_predict = model.predict(X_test)

# Make the CSV files to upload

In [34]:
output_file = pd.DataFrame({'Row': range(1,501), 'Label':y_predict})
output_file.to_csv('SVM_.csv',index = False)